# 1

Выберите задачи, в которых обязательно нужно применять поправку на множественную проверку гипотез.

* Попарное сравнение средних большого количества выборок
* Сравнение эффективности лечения пациентов в подгруппах по большому количеству признаков
* Локализация различий в активности мозга испытуемых в разных экспериментальных условиях
* Выбор инвестиционных фондов с помощью сравнения доходности каждого из них с доходностью базового актива по нескольким историческим периодам

# 2 & 3 & 4

Классификатор C4.5 и три его модификации: с оптимизацией гиперпараметра m, гиперпараметра cf и с одновременной оптимизацией обоих гиперпараметров. Эти четыре классификатора сравнивались на 14 наборах данных. На каждом датасете был посчитан AUC каждого классификатора. Данные записаны в файле:

Используя критерий знаковых рангов, проведите попарное сравнение каждого классификатора с каждым. Выберите два классификатора, различие между которыми наиболее статистически значимо.

In [36]:
from itertools import combinations

import pandas
from scipy import stats
from statsmodels.stats.multitest import multipletests

In [4]:
aucs = pandas.read_table('AUCs.txt', sep='\t', index_col=0)
aucs

,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
adult (sample),0.763,0.768,0.771,0.798
breast cancer,0.599,0.591,0.590,0.569
breast cancer wisconsin,0.954,0.971,0.968,0.967
cmc,0.628,0.661,0.654,0.657
ionosphere,0.882,0.888,0.886,0.898
iris,0.936,0.931,0.916,0.931
liver disorders,0.661,0.668,0.609,0.685
lung cancer,0.583,0.583,0.563,0.625
lymphography,0.775,0.838,0.866,0.875
mushroom,1.000,1.000,1.000,1.000


In [32]:
pairs = list(combinations(aucs, 2))
comparison = pandas.concat([
    pandas.Series([f'{column_1} vs {column_2}' for column_1, column_2 in pairs]),
    pandas.DataFrame([stats.wilcoxon(aucs[column_1], aucs[column_2]) for column_1, column_2 in pairs]),
], axis=1).sort_values('pvalue')
comparison

,0,statistic,pvalue
0,C4.5 vs C4.5+m,6.5,0.010757
2,C4.5 vs C4.5+m+cf,11.0,0.015906
5,C4.5+cf vs C4.5+m+cf,10.0,0.022909
3,C4.5+m vs C4.5+cf,17.0,0.046333
4,C4.5+m vs C4.5+m+cf,22.0,0.327826
1,C4.5 vs C4.5+cf,43.0,0.861262


Сколько статистически значимых на уровне 0.05 различий мы обнаружили?

In [33]:
comparison[comparison['pvalue'] < 0.05]

,0,statistic,pvalue
0,C4.5 vs C4.5+m,6.5,0.010757
2,C4.5 vs C4.5+m+cf,11.0,0.015906
5,C4.5+cf vs C4.5+m+cf,10.0,0.022909
3,C4.5+m vs C4.5+cf,17.0,0.046333


Судя по данным из предыдущего опроса, настройка какого из параметров классификатора даёт более значимое увеличение качества?

* `m`

# 5

Сравнивая 4 классификатора между собой, мы проверили 6 гипотез. Давайте сделаем поправку на множественную проверку. Начнём с метода Холма. Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки этим методом?

In [39]:
reject, *_ = multipletests(comparison['pvalue'], method='holm')
reject.sum()

0

# 6

Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки методом Бенджамини-Хохберга?

In [40]:
reject, *_ = multipletests(comparison['pvalue'], method='fdr_bh')
reject.sum()

3

# 7

Насколько корректно, на ваш взгляд, применение метода Бенджамини-Хохберга в этой задаче?

* Некорректно: статистики, соответствующие разным гипотезам, зависимы, так что предположения метода Бенджамини-Хохберга не выполняются

# 8

Мы подозреваем, что в проведённом с C4.5 эксперименте на самом деле классификаторы сильнее отличаются друг от друга, просто нам не удалось это заметить. Что можно сделать, чтобы увеличить вероятность обнаружения различий, если они действительно существуют?

* Взять больше датасетов